In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
books=pd.read_csv('/kaggle/input/7k-books-with-metadata/books.csv')

In [3]:
books.head(2)

In [4]:
books=books.drop(columns=['isbn10','subtitle','published_year','num_pages','ratings_count','thumbnail'])

In [5]:
books.shape

In [6]:
books.isnull().sum()

In [7]:
books.dropna(inplace=True)

In [8]:
books.duplicated().sum()

In [9]:
books.shape

In [10]:
books=books.reset_index(drop=True)
books.head(2)

In [11]:
books['categories'] = books['categories'].apply(lambda x:x.split())
books['description'] = books['description'].apply(lambda x:x.split())
books['authors'] = books['authors'].apply(lambda x:x.replace(" ",'').split())

In [12]:
books.head(2)

In [13]:
books['tags']=books['description']+books['authors']+books['categories']

In [14]:
books['tags'][0]

In [15]:
books['tags']=books['tags'].apply(lambda x: " ".join(x))
books['tags'][0]

In [16]:
df=books.drop(columns=['authors','categories','description'],axis=1)
df

In [17]:
x=df['tags'].values.tolist()
y=''
for i in x:
    y=y+i
len(y)

In [18]:
len(set(y.split()))

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=50000,stop_words='english')

In [20]:
vector = cv.fit_transform(df['tags']).toarray()

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)

In [22]:
similarity.shape

In [23]:
def recommend(book):
        index = df[df['title']== book].index[0]
        distances = sorted(list(enumerate(similarity[index])),reverse=True,key= lambda x: x[1])
        for i in distances[1:6]:
            b=df.iloc[i[0]].title
            print(b)

In [24]:
recommend('Harry Potter')

In [25]:
df[df['title']== 'Insights']

In [26]:
df[df['average_rating']>=4.74].shape

In [27]:
    for i in range(8):
        index=books[books['average_rating']==5].index[i]
        print(books[books['average_rating']==5].index[i])
        print(books.iloc[index].isbn13)
        print(books.iloc[index].title)

In [28]:
import pickle

In [29]:
pickle.dump(df,open('book_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))